## Plotting stars on StarTracker images - 18-Nov-22

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
%matplotlib inline

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver

In [ ]:
def get_catalog():
    # There's an existing catalog of stars down to 6th magnitude
    from lsst.ts.observatory.control.utils import get_catalogs_path
    from os.path import splitext
    from astropy.table import Row, Table
    catalogs = [splitext(file_name.name)[0]
                    for file_name in get_catalogs_path().glob("*.pd")]
    catalog_name = catalogs[0]
    print(catalog_name)
    catalog = Table.read(get_catalogs_path() / f"{catalog_name}.pd",
                format="pandas.json")
    catalog_coordinates = SkyCoord(
                Angle(catalog["RA"], unit=u.hourangle),
                Angle(catalog["DEC"], unit=u.deg),
                frame="icrs")    
    return catalog, catalog_coordinates

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

catalog, catalog_coordinates = get_catalog()
path = '/project/GenericCamera/101/2022/11/15/'

In [ ]:
# Get the image and solve for the WCS
seqNum = 28
filename = path + f'GC101_O_20221115_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)
wcs = genericCameraHeaderToWcs(exp)
exp.setWcs(wcs)
results = runImchar(exp, 5, 25)
solverResults = solver.run(exp, results.sourceCat, silent=True)
newWcs = solverResults.wcs
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()
   
# Now find objects within the image    
skyLocation = SkyCoord(raSolve*u.deg, decSolve*u.deg)
d2d = skyLocation.separation(catalog_coordinates)
catalogmask = d2d < 3.0 * u.deg # Chose 3 degrees because the image is roughly 6 degrees across
list = catalog_coordinates[catalogmask]
xs = []
ys = []
names = []
for item in list:
    spherePoint = SpherePoint(afwAngle(item.ra.rad), afwAngle(item.dec.rad))
    point = newWcs.skyToPixel(spherePoint)
    xs.append(point.x)
    ys.append(point.y)
    match = item.match_to_catalog_sky(catalog_coordinates)
    name = catalog[match[0]][0]
    names.append(name)

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field = 20221115 seqNo 28")
img = plt.imshow(exp.image.array,  interpolation='Nearest', cmap='gray')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, name in zip(xs, ys, names):
    plt.text(x+50,y, name, color='g')
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Labels_20221115_28.pdf")